In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import pipeline
import tweepy
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
import nltk
from utils import numerical_df
from baselines import xgboost_baseline, majority_class_baseline, random_class_baseline, ibm_baseline, bert_baseline, bm25_baseline, qa_model_baseline
from sklearn.metrics import f1_score as f1, precision_score as ps, recall_score as rs
from sklearn.model_selection import KFold
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer, util

import emoji

### Hyperparameters

In [2]:
df_arg = numerical_df(pd.read_csv('./mkturk/argumentative.csv')).drop(columns = ['Unnamed: 0'])
df_claim = numerical_df(pd.read_csv('./mkturk/claims-large.csv'))
df_evidence = numerical_df(pd.read_csv('./mkturk/evidence.csv'))
## Remove empty strings
#df = df[df.tweet != '']
#df = df[df.tweet.notnull()]

/home/mah/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [3]:
len(df_arg[df_arg.argumentative > 0.5])

361

In [4]:
df_arg.loc[df_arg.argumentative > 0.49, 'argumentative'] = 1
df_arg.loc[df_arg.argumentative < 0.5, 'argumentative'] = 0

df_claim.loc[df_claim.claim >= 0.2, 'claim'] = 1
df_claim.loc[df_claim.claim < 0.2, 'claim'] = 0


df_evidence.loc[df_evidence.evidence < 0.2, 'evidence'] = 0
df_evidence.loc[df_evidence.evidence >= 0.2, 'evidence'] = 1


len(df_arg), len(df_claim), len(df_evidence)

(600, 2985, 2985)

In [5]:
sum(df_arg.argumentative), sum(df_claim.claim), sum(df_evidence.evidence)

(390.0, 785.0, 833.0)

In [6]:
current_classes = ['argumentative', 'claim', 'evidence']

In [7]:
display(majority_class_baseline(df_arg, ['argumentative']))
display(majority_class_baseline(df_claim, ['claim']))
display(majority_class_baseline(df_evidence, ['evidence']))

,Task,F1,Precision,Recall
0,argumentative,0.6499999999999999,0.6499999999999999,0.6499999999999999


,Task,F1,Precision,Recall
0,claim,0.7370261049134699,0.7370261049134699,0.7370261049134699


,Task,F1,Precision,Recall
0,evidence,0.7209400462391417,0.7209400462391417,0.7209400462391417


In [8]:
display(random_class_baseline(df_arg, ['argumentative']))
display(random_class_baseline(df_claim, ['claim']))
display(random_class_baseline(df_evidence, ['evidence']))

,Task,F1,Precision,Recall
0,argumentative,0.6629485813464142,0.6649277749745595,0.6690377604269439


,Task,F1,Precision,Recall
0,claim,0.2870442045861247,0.28791483214367797,0.2894379137691777


,Task,F1,Precision,Recall
0,evidence,0.24385208107944978,0.24572947809007828,0.24382800327329637


In [10]:
#display(xgboost_baseline(df_arg, tasks = ['argumentative'], use_topic = False))
df = df_claim.reset_index(drop=True)
df['tweet'] =  df.tweet.map(lambda x: x.replace('<MENTION>', '@user'))
display(xgboost_baseline(df, fold = 3, model_name = 'cardiffnlp/twitter-roberta-base', tasks = ['claim'], average='macro'))

Loading language model
2022-05-16 15:18:23,519 No model_max_length in Tokenizer's config.json - setting it to 512. Specify desired model_max_length by passing it as attribute to embedding instance.
Generating the embeddings
Generating results for claim
Fitting 3 folds for each of 312 candidates, totalling 936 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    3.5s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  2.2min


KeyboardInterrupt: 

In [ ]:

df = df_evidence.reset_index(drop=True)
df['tweet'] =  df.tweet.map(lambda x: x.replace('<MENTION>', '@user'))
display(xgboost_baseline(df, fold = 3, model_name = 'cardiffnlp/twitter-roberta-base', tasks = ['evidence'], average='macro'))

In [9]:
#bm25_baseline(df, current_classes)
#display(bm25_baseline(df_arg, ['argumentative']))
display(bm25_baseline(df_claim, ['claim'], average='macro'))
display(bm25_baseline(df_evidence, ['evidence'], average='macro'))

,Task,F1,Precision,Recall
0,claim,0.57631,0.608771,0.585741


,Task,F1,Precision,Recall
0,evidence,0.57774,0.604023,0.585113


In [ ]:
qa_model_baseline(df_claim.copy(), ['claim'], model_cutoff = 0.5, average='macro')

In [ ]:
#display(ibm_baseline(df_arg, ['argumentative']))
#display(ibm_baseline(df_claim, ['claim'], average='macro'))
display(ibm_baseline(df_evidence, ['evidence'], average='macro'))

In [ ]:
#bert_baseline(df, tasks=current_classes, use_topic=True, batch_size=5, epochs = 1)
#display(ibm_baseline(df_arg, ['argumentative']))
df = df_claim.reset_index(drop=True)
df['tweet'] =  df.tweet.map(lambda x: x.replace('<MENTION>', '@user'))#.map(emoji.demojize)
display(bert_baseline(df, fold = 3, model_name = 'cardiffnlp/twitter-roberta-base', tasks = ['claim'], learning_rate = 0.000001, use_topic=True, batch_size=5, epochs = 15))

In [ ]:
df = df_arg.reset_index(drop=True)
df['tweet'] =  df.tweet.map(lambda x: x.replace('<MENTION>', '@USER'))
display(bert_baseline(df, model_name = 'cardiffnlp/twitter-roberta-base', fold = 10, tasks = ['argumentative'], learning_rate = 0.000005, use_topic=False, batch_size=5, epochs = 5))